In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from utils import *

In [3]:
INPUT_PATH = "../data/base.parquet"
MODEL_PATH = "https://tfhub.dev/google/universal-sentence-encoder/4"

In [4]:
df = pd.read_parquet(INPUT_PATH)

In [5]:
df.head()

,community,user_id,thread_id,date,subject,category,body,quotes,uuid
0,Evolution Forums,b3d65d87887506a226923f1b08b67c69faba1efb26a2fa...,37163,2014-12-10 22:59:52,Changes to order process for FE listings (Auto...,Announcements,Henceforth Evolution will automatically releas...,nan,-4170171173165973422
1,Evolution Forums,046ecde3177c87e9173e3d08ec5000ccf13a6f087e1929...,37163,2014-12-10 23:03:32,Re: Changes to order process for FE listings (...,Announcements,Sweet! No need to keep logging on in the morni...,nan,-4410324336093370437
2,Evolution Forums,a4d9a8a70225b297661655a7ea032387ae4b2fdc5df5b5...,37163,2014-12-10 23:08:15,Re: Changes to order process for FE listings (...,Announcements,"Good move, although it will be interesting to ...",nan,2292277039528005701
3,Evolution Forums,753cece12915dd5e69d190c52917990a43a76b07e2f461...,37163,2014-12-10 23:09:08,Re: Changes to order process for FE listings (...,Announcements,TOOP! thanks Kimble This speeds up the work fo...,nan,6452620963465746902
4,Evolution Forums,5f1f3b0a3a79ea76b1e3b1c5df4ba05d2a9f2c8b30459b...,37163,2014-12-10 23:09:34,Re: Changes to order process for FE listings (...,Announcements,"well done, you are all doing a great work kimble",nan,-7115332770257554039


In [6]:
!nvidia-smi

Mon Feb  8 22:13:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 450.57       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:C1:00.0  On |                  N/A |
| 41%   46C    P3    52W / 280W |  22954MiB / 24217MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Embedding the *Body* with USE4

In [ ]:
embed = hub.load(MODEL_PATH)

In [7]:
column_to_embed = "body"
body_embeddings = tf_hub_embedder(df, column_to_embed, embed)

  0%|          | 0/6355 [00:00<?, ?it/s]

In [9]:
%%time
with open(f'../data/{column_to_embed}_embeddings.npy', 'wb') as f:
    np.save(f, body_embeddings)

CPU times: user 381 ms, sys: 4 s, total: 4.38 s
Wall time: 6.64 s


# Embedding the *Subject* with USE4

In [11]:
column_to_embed = "subject"
subject_embeddings = tf_hub_embedder(df, column_to_embed, embed)

  0%|          | 0/6355 [00:00<?, ?it/s]

In [12]:
%%time
with open(f'../data/{column_to_embed}_embeddings.npy', 'wb') as f:
    np.save(f, subject_embeddings)

CPU times: user 1.82 s, sys: 3.56 s, total: 5.39 s
Wall time: 13.1 s


# Embedding the *Category* with USE4

In [13]:
column_to_embed = "category"
category_embeddings = tf_hub_embedder(df, column_to_embed, embed)

  0%|          | 0/6355 [00:00<?, ?it/s]

In [14]:
%%time
with open(f'../data/{column_to_embed}_embeddings.npy', 'wb') as f:
    np.save(f, category_embeddings)

CPU times: user 2.17 s, sys: 4.88 s, total: 7.05 s
Wall time: 16.1 s


# Embedding the *Body* with BERTweet

In [6]:
bertweet_tokenizer = BertweetTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
bertweet_model = TFAutoModel.from_pretrained("vinai/bertweet-base")
bertweet_model.trainable=False

Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [8]:
%%time
body_bertweet_embeddings = huggingface_embedder(df, 'body', bertweet_tokenizer, bertweet_model)

  0%|          | 0/6355 [00:00<?, ?it/s]

CPU times: user 4h 10min 4s, sys: 1h 10min, total: 5h 20min 5s
Wall time: 4h 31min 30s


In [9]:
%%time
with open(f'../data/body_bertweet_embeddings.npy', 'wb') as f:
    np.save(f, body_bertweet_embeddings)

CPU times: user 931 ms, sys: 7.09 s, total: 8.02 s
Wall time: 12.2 s
